In [3]:
import numpy as np
import pandas as pd

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
vehicles_list = ['ICEV', 'PHEV', 'BEV', 'FCEV']
demand_list = ['normal','EU100','EU0']

In [3]:
# SECOND PHASE parsing
# translate the outlook into output requirements
file_path = "data/outlook_vehicles_sales.csv"
outlook = pd.read_csv(file_path)
outlook = pd.melt(outlook, id_vars=['category','region','year'], value_vars=vehicles_list,
                    var_name='vehicles', value_name='outlook')
outlook.drop(columns="region",inplace=True)

In [4]:
file_path = "inputs/input_manufacturing.csv"
requirement = pd.read_csv(file_path)
requirement = pd.melt(requirement, id_vars=['Product'], value_vars=vehicles_list,
                    var_name='vehicles', value_name='requirement')
requirement.rename(columns={'Product': 'sector'},inplace=True)

In [5]:
file_path = "inputs/input_market_shares.csv"
market_shares = pd.read_csv(file_path)
market_shares = pd.melt(market_shares, id_vars=["scenario","region"],var_name="vehicles",value_vars=vehicles_list, value_name="shares")
market_shares.rename(columns={"scenario":"demand"},inplace=True)

In [6]:
outlook_requirement = pd.merge(outlook, requirement, on='vehicles', how='outer')
outlook_requirement_shares = pd.merge(market_shares, outlook_requirement, on='vehicles', how='outer')
outlook_requirement_shares["value"] = outlook_requirement_shares["outlook"] * outlook_requirement_shares["requirement"] * outlook_requirement_shares["shares"]

In [7]:
outlook_sectors = outlook_requirement_shares.pivot_table(index=['category','demand','region', 'year', 'sector'],
                          columns='vehicles',
                          values='value',
                          aggfunc='sum')

In [8]:
scenario_list = ["Projection-APS","Projection-STEPS"]
year_list = [2023,2025,2030,2035]

for scenario in scenario_list:
    for demand in demand_list: 
        outlook_scenario = outlook_sectors.reset_index()
        outlook_scenario = outlook_scenario[(outlook_scenario.category==scenario)&(outlook_scenario.demand==demand)]
        outlook_scenario = outlook_scenario[["region","year","sector","ICEV","PHEV","BEV","FCEV"]]
        
        file_name = "data/outlook_manufacturing_"+scenario+"_"+demand+".csv"
        outlook_scenario.to_csv(file_name,index=False)

In [9]:








#SPACER

In [5]:
# PER COUNTRY PHASE parsing
# translate the outlook into output requirements
file_path = "data/outlook_vehicles_sales.csv"
outlook = pd.read_csv(file_path)
outlook = pd.melt(outlook, id_vars=['category','region','year'], value_vars=vehicles_list,
                    var_name='vehicles', value_name='outlook')
outlook.drop(columns="region",inplace=True)

In [6]:
file_path = "inputs/input_manufacturing.csv"
requirement = pd.read_csv(file_path)
requirement = pd.melt(requirement, id_vars=['Product'], value_vars=vehicles_list,
                    var_name='vehicles', value_name='requirement')
requirement.rename(columns={'Product': 'sector'},inplace=True)

In [7]:
file_path = "inputs/input_market_shares_country.csv"
market_shares = pd.read_csv(file_path)
market_shares = pd.melt(market_shares, id_vars=["scenario","region"],var_name="vehicles",value_vars=vehicles_list, value_name="shares")
market_shares.rename(columns={"scenario":"demand"},inplace=True)

In [8]:
market_shares = market_shares.groupby(["demand","region","vehicles"])["shares"].sum().reset_index()
market_shares

,demand,region,vehicles,shares
0,normal,CN,BEV,0.1275
1,normal,CN,FCEV,0.1275
2,normal,CN,ICEV,0.1275
3,normal,CN,PHEV,0.1275
4,normal,CZ,BEV,0.0612
5,normal,CZ,FCEV,0.0612
6,normal,CZ,ICEV,0.0612
7,normal,CZ,PHEV,0.0612
8,normal,DE,BEV,0.2545
9,normal,DE,FCEV,0.2545


In [9]:
# calculate only for one demand
requirement_shares = pd.merge(market_shares, requirement, on='vehicles', how='outer')
requirement_shares["value"] = requirement_shares["requirement"]

requirement_shares.drop(columns=["demand","shares","requirement"]).to_csv("data/outlook_footprint_manufacturing_country.csv",index=False)

In [17]:
# join the data : outlook, requirement, and market shares
outlook_requirement = pd.merge(outlook, requirement, on='vehicles', how='outer')
outlook_requirement_shares = pd.merge(market_shares, outlook_requirement, on='vehicles', how='outer')
outlook_requirement_shares["value"] = outlook_requirement_shares["outlook"] * outlook_requirement_shares["requirement"] * outlook_requirement_shares["shares"]
outlook_requirement_shares

,demand,region,vehicles,shares,category,year,outlook,sector,requirement,value
0,normal,CN,BEV,0.1275,Projection-APS,2020,760000.0,Crude petroleum and services related to crude ...,3.300000e-07,0.031977
1,normal,CN,BEV,0.1275,Projection-APS,2020,760000.0,Textiles (17),5.602000e-06,0.542834
2,normal,CN,BEV,0.1275,Projection-APS,2020,760000.0,Wearing apparel; furs (18),1.616250e-05,1.566146
3,normal,CN,BEV,0.1275,Projection-APS,2020,760000.0,Leather and leather products (19),7.352000e-07,0.071241
4,normal,CN,BEV,0.1275,Projection-APS,2020,760000.0,Wood and products of wood and cork (except fur...,3.439000e-07,0.033324
...,...,...,...,...,...,...,...,...,...,...
28723,normal,US,PHEV,0.1898,Projection-STEPS,2035,620000.0,Computer and related services (72),1.763729e-04,20.754857
28724,normal,US,PHEV,0.1898,Projection-STEPS,2035,620000.0,Other business services (74),1.048922e-03,123.432981
28725,normal,US,PHEV,0.1898,Projection-STEPS,2035,620000.0,Education services (80),1.618640e-05,1.904751
28726,normal,US,PHEV,0.1898,Projection-STEPS,2035,620000.0,Membership organisation services n.e.c. (91),1.359680e-05,1.600017


In [18]:
outlook_sectors = outlook_requirement_shares.pivot_table(index=['category','demand','region', 'year', 'sector'],
                          columns='vehicles',
                          values='value',
                          aggfunc='sum')

outlook_sectors

vehicles                                                                                       BEV  \
category         demand region year sector                                                           
Projection-APS   normal CN     2020 Air transport services (62)                           4.799360   
                                    Aviation Gasoline                                     0.000000   
                                    Basic iron and steel and of ferro-alloys and fi...   28.126456   
                                    Bitumen                                               0.025804   
                                    Chemicals nec                                         3.754807   
...                                                                                            ...   
Projection-STEPS normal US     2035 Textiles (17)                                        13.822375   
                                    Wearing apparel; furs (18)                           39.879353   
                                    White Spirit & SBP                                    0.056997   
                                    Wholesale trade and commission trade services, ...  340.146388   
                                    Wood and products of wood and cork (except furn...    0.848539   

vehicles                                                                                    FCEV  \
category         demand region year sector                                                         
Projection-APS   normal CN     2020 Air transport services (62)                         0.001175   
                                    Aviation Gasoline                                   0.000000   
                                    Basic iron and steel and of ferro-alloys and fi...  0.012780   
                                    Bitumen                                             0.000008   
                                    Chemicals nec                                       0.348032   
...                                                                                          ...   
Projection-STEPS normal US     2035 Textiles (17)                                       0.004962   
                                    Wearing apparel; furs (18)                          0.014229   
                                    White Spirit & SBP                                  0.000014   
                                    Wholesale trade and commission trade services, ...  0.122099   
                                    Wood and products of wood and cork (except furn...  0.000305   

vehicles                                                                                      ICEV  \
category         demand region year sector                                                           
Projection-APS   normal CN     2020 Air transport services (62)                           2.821380   
                                    Aviation Gasoline                                     0.000000   
                                    Basic iron and steel and of ferro-alloys and fi...  466.553504   
                                    Bitumen                                               0.140643   
                                    Chemicals nec                                        62.283654   
...                                                                                            ...   
Projection-STEPS normal US     2035 Textiles (17)                                         2.165505   
                                    Wearing apparel; furs (18)                            6.169448   
                                    White Spirit & SBP                                    0.002938   
                                    Wholesale trade and commission trade services, ...   53.289601   
                                    Wood and products of wood and cork (except furn...    0.132938   

vehicles                                     

In [20]:
scenario_list = ["Projection-APS","Projection-STEPS"]
year_list = [2023,2025,2030,2035]
demand_list = ['normal']

for scenario in scenario_list:
    for demand in demand_list:
        outlook_scenario = outlook_sectors.reset_index()
        outlook_scenario = outlook_scenario[(outlook_scenario.category==scenario)&(outlook_scenario.demand==demand)]
        outlook_scenario = outlook_scenario[["region","year","sector","ICEV","PHEV","BEV","FCEV"]]

        file_name = "data/outlook_manufacturing_country_"+scenario+"_"+demand+".csv"
        outlook_scenario.to_csv(file_name,index=False)